# BatDetect
## Species specific classifications of bats from acoustic data ##


*GPU is automatically enabled within the script* \
Classification time: 
* 17 seconds per minute audio with 4 CPUS and no GPU
* 5 seconds with 4 CPUS and 1 GPU 
* 7 seconds with 8 CPUS and 1 GPU 
* 6 seconds with 8 CPUS and 2 GPU 


## Mount buckets ##

In [1]:
!mountpoint -q /home/jupyter/gcs_outputs && echo "mounted" || mkdir -p gcs_outputs; gcsfuse --implicit-dirs --rename-dir-limit=100 --disable-http2 --max-conns-per-host=100 "acoustic-processing-outputs" "/home/jupyter/gcs_outputs"
!mountpoint -q /home/jupyter/gcs_raw && echo "mounted" || mkdir -p gcs_raw; gcsfuse --implicit-dirs --rename-dir-limit=100 --disable-http2 --max-conns-per-host=100 "acoustic-data-raw" "/home/jupyter/gcs_raw"

2022/11/25 10:00:01.762842 Start gcsfuse/0.41.8 (Go version go1.18.4) for app "" using mount point: /home/jupyter/gcs_outputs
2022/11/25 10:00:01.790384 Opening GCS connection...
2022/11/25 10:00:01.944621 Mounting file system "acoustic-processing-outputs"...
2022/11/25 10:00:01.968268 File system has been successfully mounted.
2022/11/25 10:00:02.100917 Start gcsfuse/0.41.8 (Go version go1.18.4) for app "" using mount point: /home/jupyter/gcs_raw
2022/11/25 10:00:02.113611 Opening GCS connection...
2022/11/25 10:00:02.219514 Mounting file system "acoustic-data-raw"...
2022/11/25 10:00:02.234823 File system has been successfully mounted.


In [3]:
![ -d "/home/jupyter/batdetect_v3-master" ] && echo "Scripts are downloaded" || !gcloud storage cp -r 'gs://data-processing-scripts/batdetect_v3-master/' .

Scripts are downloaded


In [4]:
import os
path = "/home/jupyter/batdetect_v3-master"
os.chdir(path)

In [5]:
# import the necessary libraries
import glob
import config
import matplotlib.pyplot as plt

import bat_detect.utils.detector_utils as du
import bat_detect.utils.audio_utils as au
import bat_detect.utils.plot_utils as viz
import time

In [5]:
# setup the arguments
args = {}

## Set input and output dirs below ##

In [6]:
args['ann_dir'] = "/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/"
args['audio_dir'] = "/home/jupyter/gcs_raw/nr-acoustic-data/bat-config/"

In [7]:
# Leave these as is
args['detection_threshold'] = 0.3
args['time_expansion_factor'] = 1
args['model_url'] = config.MODEL_URL
args['model_path'] = os.path.join('models', os.path.basename(args['model_url']))

args['cnn_features'] = False
args['spec_features'] = False
args['quiet'] = True
args['save_preds_if_empty'] = False
args['spec_slices'] = False
args['chunk_size'] = 3
args['save_preds_if_empty'] = True

NameError: name 'config' is not defined

In [ ]:
# load the model
model, params = du.load_model(args['model_url'], args['model_path'])

# load files
files = du.get_audio_files(args['audio_dir'] )
error_files = []


# check if results file already exists - skip
for ii, audio_file in enumerate(files):
    msd = os.path.basename(os.path.dirname(audio_file))
    project = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(audio_file))))
    results_file = os.path.join(args['ann_dir'], project, msd, str(os.path.basename(audio_file) + '.json'))
    
    if os.path.isfile(results_file):
        print(results_file)
        print("File exists")

        
    else:
        t1 = int(time.time())
        try:
            results = du.process_file(audio_file, model, params, args)
            if args['save_preds_if_empty'] or (len(results['pred_dict']['annotation']) > 0):
                msd = os.path.basename(os.path.dirname(audio_file))
                project = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(audio_file))))
                results_path = os.path.join(args['ann_dir'], project, msd, os.path.basename(audio_file))
                print("Saving results to_" + results_path)
                du.save_results_to_file(results, results_path)
        except:
            error_files.append(audio_file)
            print("Error processing file!")
            
        t2 = int(time.time())
        print('File processed in ' + str(t2 - t1) + ' seconds')
    

# Shutdown the notebook   
!sudo shutdown -h now 

/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210000.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210100.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210200.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210300.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210400.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210500.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210600.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/M13_96_bat/20220325_210700.WAV.json
File exists
/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-

/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds


/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds


...

Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds


/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds


/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Error processing file!
File processed in 0 seconds
Saving results to_/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/MSD-43/20220302_210000.WAV
File processed in 5 seconds
Saving results to_/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/MSD-43/20220302_210100.WAV
File processed in 5 seconds
Saving results to_/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/MSD-43/20220302_210200.WAV
File processed in 5 seconds
Saving results to_/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/MSD-43/20220302_210300.WAV
File processed in 6 seconds
Saving results to_/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acoustic-data/MSD-43/20220302_210400.WAV
File processed in 5 seconds
Saving results to_/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/nr-acousti

In [ ]:
# print summary info for the individual detections 
print('Results for ' + results['pred_dict']['id'])
print('{} calls detected\n'.format(len(results['pred_dict']['annotation'])))

print('time\tprob\tlfreq\tspecies_name')
for ann in results['pred_dict']['annotation']:
    print('{}\t{}\t{}\t{}'.format(ann['start_time'], ann['class_prob'], ann['low_freq'], ann['class']))

Results for 20220310_214100.WAV
33 calls detected

time	prob	lfreq	species_name
0.9175	0.227	24609	Plecotus austriacus
0.9825	0.286	23750	Plecotus austriacus
1.2445	0.281	24609	Plecotus austriacus
1.3115	0.195	26328	Plecotus austriacus
2.4235	0.211	24609	Plecotus austriacus
7.1445	0.233	24609	Plecotus austriacus
7.7325	0.259	25468	Plecotus austriacus
7.9305	0.277	23750	Plecotus austriacus
9.7645	0.316	58125	Pipistrellus pygmaeus
10.2235	0.33	58984	Pipistrellus pygmaeus
10.5515	0.24	23750	Plecotus austriacus
11.1415	0.305	24609	Plecotus austriacus
12.3205	0.302	58984	Pipistrellus pygmaeus
14.4165	0.295	58984	Pipistrellus pygmaeus
19.5955	0.31	58125	Pipistrellus pygmaeus
19.6605	0.369	58125	Pipistrellus pygmaeus
22.3485	0.353	58984	Pipistrellus pygmaeus
23.1995	0.271	23750	Plecotus austriacus
26.0165	0.218	25468	Plecotus austriacus
26.2135	0.314	24609	Plecotus austriacus
26.4765	0.24	25468	Plecotus austriacus
38.1415	0.27	26328	Plecotus austriacus
38.2075	0.308	24609	Plecotus austriacus


## Concatenate results into one csv ##

In [8]:
import glob as glob
import pandas as pd
import os
from datetime import date

In [9]:
args['ann_dir']

'/home/jupyter/gcs_outputs/nr-acoustic-data/batdetect/'

In [29]:
d = date.today()
d = str(d)
results_list = glob.glob(args['ann_dir'] +  '*/*/*.csv')
metadata = pd.read_csv('/home/jupyter/gcs_raw/nr-acoustic-data/metadata/NR_deployment_2022_ARUs.csv')

li = []

for filename in results_list:
    df = pd.read_csv(filename)
    df['file_name'] = os.path.splitext(os.path.basename(filename))[0]
    df['path'] = filename
    df['date'] = [f[0:7] for f in df['file_name']]
    df['ID'] = os.path.basename(os.path.dirname(filename))
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame = pd.merge(frame, metadata, on='ID', how='inner')
frame.to_csv(args['ann_dir'] + d + '_concatenated_results_bat_detect.csv')

In [30]:
frame

,id,det_prob,start_time,end_time,high_freq,low_freq,class,class_prob,file_name,path,...,SD card,Config,Housing,Site area,LocationID,lat,long,Date deployed,Date collected,Data saved
0,0,0.508,0.0935,0.1012,62871,47812,Pipistrellus pipistrellus,0.477,20220325_210000.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,M-13,bat_50_sec.config,AudioMoth case w Schlegel vent,Barnes,NaN,51.46957198,-0.248539788,25/03/2022,NaN,y
1,1,0.479,0.1865,0.1976,63606,47812,Pipistrellus pipistrellus,0.445,20220325_210000.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,M-13,bat_50_sec.config,AudioMoth case w Schlegel vent,Barnes,NaN,51.46957198,-0.248539788,25/03/2022,NaN,y
2,2,0.503,0.2855,0.2950,62486,47812,Pipistrellus pipistrellus,0.477,20220325_210000.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,M-13,bat_50_sec.config,AudioMoth case w Schlegel vent,Barnes,NaN,51.46957198,-0.248539788,25/03/2022,NaN,y
3,3,0.471,0.3735,0.3838,62890,47812,Pipistrellus pipistrellus,0.443,20220325_210000.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,M-13,bat_50_sec.config,AudioMoth case w Schlegel vent,Barnes,NaN,51.46957198,-0.248539788,25/03/2022,NaN,y
4,4,0.459,0.4695,0.4810,63016,46953,Pipistrellus pipistrellus,0.406,20220325_210000.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,M-13,bat_50_sec.config,AudioMoth case w Schlegel vent,Barnes,NaN,51.46957198,-0.248539788,25/03/2022,NaN,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523334,0,0.319,17.7185,17.7283,31475,25468,Eptesicus serotinus,0.168,20220311_221000.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,MSD-43,bat_2100_2300.config,AudioMoth case w Schlegel vent,Devonshire Road Nature Reserve,NaN,51.44503,-0.04977,03/02/2022,25/03/2022,y
523335,0,0.327,30.5635,30.5729,32066,25468,Eptesicus serotinus,0.187,20220311_221300.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,MSD-43,bat_2100_2300.config,AudioMoth case w Schlegel vent,Devonshire Road Nature Reserve,NaN,51.44503,-0.04977,03/02/2022,25/03/2022,y
523336,0,0.362,6.1845,6.1967,30192,26328,Nyctalus leisleri,0.245,20220311_221400.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,MSD-43,bat_2100_2300.config,AudioMoth case w Schlegel vent,Devonshire Road Nature Reserve,NaN,51.44503,-0.04977,03/02/2022,25/03/2022,y
523337,0,0.345,20.8655,20.8765,31065,26328,Nyctalus leisleri,0.204,20220311_221500.WAV,/home/jupyter/gcs_outputs/nr-acoustic-data/bat...,...,MSD-43,bat_2100_2300.config,AudioMoth case w Schlegel vent,Devonshire Road Nature Reserve,NaN,51.44503,-0.04977,03/02/2022,25/03/2022,y
